In [26]:
import wrds
import pandas as pd
from pathlib import Path

In [27]:
file_path = Path.cwd() / "SAMPLE_10Ks"
file_path.mkdir(parents=True, exist_ok=True)


In [28]:
def create_codes(path):
    names_list = []
    cik_list = []
    year_list = []

    for files in path.glob("*.txt"):
        file = files.read_text().splitlines()
        loop = -1
        for lines in file:
            loop = loop + 1
            line = lines.rfind("COMPANY CONFORMED NAME:")
            if line != -1:
                name = file[loop].replace("COMPANY CONFORMED NAME:", "").strip().rstrip()
                names_list.append(name)
                break

    for files in path.glob("*.txt"):
        file = files.read_text().splitlines()
        year = files.name[:-5:-1]
        year = year[::-1]
        year_list.append(year)
        loop = -1
        for lines in file:
            loop = loop + 1
            line = lines.rfind("CENTRAL INDEX KEY:")
            if line != -1:
                cik = file[loop].replace("CENTRAL INDEX KEY:", "").strip().rstrip()
                cik_list.append(cik)
                break
    
    
    company_info = pd.DataFrame()
    company_info["Companies"] = names_list
    company_info["CIK"] = cik_list
    company_info["Year"] = year_list

    return company_info

In [29]:
company_info = create_codes(file_path)

company_info.drop_duplicates()

year_list = list(company_info["Year"])
cik_list = list(company_info["CIK"])
names_list = list(company_info["Companies"])

In [30]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [31]:
tuple_cik = tuple(cik_list)
cik_params = {"cik_list": tuple_cik}

In [32]:
CIK_query = db.raw_sql("""SELECT cik, cusip, lpermno, linkdt FROM crsp_a_ccm.ccm_lookup
                 WHERE cik IN %(cik_list)s""", params = cik_params)
CUSIP = pd.DataFrame(CIK_query)

cusip_tuple = tuple(CUSIP["cusip"])
cusip_params = {"cusip_list": cusip_tuple}

permno_tuple = tuple(CUSIP["lpermno"])
permno_params = {"permno_list": permno_tuple}

In [33]:
db.close

<bound method Connection.close of <wrds.sql.Connection object at 0x137c58dd0>>

In [34]:
Returns_Query = db.raw_sql("""SELECT permno,yyyy,cusip,annret,anncap  FROM crsp_a_stock.stkannsecuritydata
                 WHERE permno IN %(permno_list)s""", params = permno_params)
RETURNS = pd.DataFrame(Returns_Query)

In [35]:
db.close

<bound method Connection.close of <wrds.sql.Connection object at 0x137c58dd0>>

In [36]:
RETURNS

,permno,yyyy,cusip,annret,anncap
0,10225,1925,None,NaN,130728.50
1,10225,1926,None,0.144335,139769.00
2,10225,1927,None,0.518574,200613.00
3,10225,1928,None,0.067666,204057.00
4,10225,1929,None,0.292785,314725.50
...,...,...,...,...,...
5651,93274,2019,16949H20,-0.130662,22867.78
5652,93274,2020,98880R10,0.158798,35185.05
5653,93274,2021,98880R10,-0.259259,35333.00
5654,93274,2022,98880R10,-0.540000,16480.88


In [38]:
import wrds

# Connect to WRDS
db = wrds.Connection()

# Specify the CIK number you want to convert
cik_number = '0000320193'  # Example CIK for Apple Inc.

# Query to find the corresponding PERMNO
query = f"""
SELECT permno, cik
FROM crsp.ccmxpf_linktable
WHERE cik = '{cik_number}'
AND lpermno IS NOT NULL
ORDER BY linkdt DESC
LIMIT 1;
"""

# Execute the query
result = db.raw_sql(query)

# Check if we found a match and print it
if not result.empty:
    print(f"CIK {cik_number} corresponds to PERMNO {int(result['permno'].iloc[0])}")
else:
    print("No matching PERMNO found for the given CIK.")


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "permno" does not exist
LINE 2: SELECT permno, cik
               ^
HINT:  Perhaps you meant to reference the column "ccmxpf_linktable.lpermno".

[SQL: 
SELECT permno, cik
FROM crsp.ccmxpf_linktable
WHERE cik = '0000320193'
AND lpermno IS NOT NULL
ORDER BY linkdt DESC
LIMIT 1;
]
(Background on this error at: https://sqlalche.me/e/20/f405)